# Live Hate Prediction

## Skalierung

Was passiert wenn die Datenmengen größenordnungsmäßig zunehmen? (1x/10x/100x... oder 2x/4x/8x/16x/...)?
- Erhöhte Verarbeitungsleistung: Sollen mehr Tweets/Sekunde live klassifiziert werden, erfordert dass eine höhere Verarbeitungsleistung, da eine größere Datenmenge in Echtzeit analysiert werden müssen. Dies erfordert möglicherweise den Einsatz von mehr oder leistungsfähigerer Hardware. Durch unsere Implementierung in der Spark Streaming Bibliothek kann eine horizontale Skalierung erreicht werden.
- Erhöhte Speicherkapazität: Die Skalierung der Live-Klassifizierung erfordert auch mehr Speicherkapazität, um die Daten zu speichern. So könnte es z.B. sinnvoll sein die Predictions zu persistieren um Transparenz zu ermöglichen. Die aktuellen Ergebnisse könnten genutzt werden, um die Trainingsdatensätze zu erweitern. Mithilfe von Systemen wie HDFS könnte eine horizontale Skalierung der Datenhaltung ermögmlicht werden.
- Erhöhte Bandbreite: Die Hochskalierung der Live-Twitter-Klassifizierung erfordert möglicherweise mehr Bandbreite für die Übertragung der zusätzlichen Daten, die analysiert werden. Dies kann zu einem Problem werden, wenn die Netzwerkinfrastruktur nicht in der Lage ist, die erhöhte Last zu bewältigen.

Was passiert wenn die Anfragen oder Abfragen größenordnungsmäßig zunehmen oder die Latenzbedingungen größenordnungsmäßig abnehmen?
- Erhöhte Verarbeitungszeit: Wenn die Anfragen oder Abfragen größer werden, kann ihre Verarbeitung länger dauern, da mehr Daten zu analysieren sind. Dies kann zu erhöhten Latenzzeiten und einer insgesamt langsameren Leistung führen.
- Erhöhte Ressourcennutzung: Größere Anfragen oder Abfragen erfordern unter Umständen auch mehr Ressourcen, z. B. Verarbeitungsleistung, Arbeitsspeicher und Speicherplatz, um verarbeitet zu werden. Dies kann zu einer erhöhten Ressourcennutzung führen und möglicherweise die Leistung anderer Aufgaben oder Prozesse, die gleichzeitig laufen, beeinträchtigen.
- Geringere Leistung: Wenn die Latenzbedingungen abnehmen, kann es schwieriger werden, Anfragen oder Abfragen in Echtzeit zu verarbeiten, da weniger Zeit für die Verarbeitung der Daten zur Verfügung steht. Dies kann zu einer verringerten Leistung und möglicherweise zu verpassten Analysemöglichkeiten führen.
- Erhöhte Komplexität: Sowohl größere Anfragen oder Abfragen als auch geringere Latenzzeiten können die Datenanalyse komplexer machen, da mehr Daten zu verarbeiten sind und weniger Zeit zur Verfügung steht. Dies kann den Einsatz mehr Hardware oder optimierteren Algorithmen erforderlich machen, um das größere Datenvolumen zu bewältigen und die Echtzeitverarbeitungsanforderungen zu erfüllen.

Welche Pfade sind einfach/schwieriger Skalierbar?
- Datenspeicherung: Einfache Skalierbarkeit, da weitere Server hinzugefügt werden können, um die erhöhte Last zu bewältigen und mehr Daten von der Twitter-API zu sammeln, zu verarbeiten oder zu speichern (z.B. Einsatz von HDFS)
- Datenvorverarbeitung: Aktueller Prozess zur Datenbereinigung ist relativ simpel (entfernen von Sonderzeichen, Kleinschreibung, etc.). Falls die Datenvorverarbeitung jedoch komplexer werden sollte (komplexere Machine Learning Algorithmen, Erkennen von Text in Bildern/Audio konvertieren in Text, etc.), könnte der Aufwand für die Vorverarbeitung deutlich steigen. Durch horizontale Skalierung mit Spark, ist die Datenverarbeitung jedoch gut skalierbar.
- Modellinterferenz: Skaliert gut mit großen Datenmengen, da das ausgewählte Modell sehr einfach ist und wenig Rechenzeit für die Vorhersage mit dem Modell benötigt wird. Die Modellinterferenz skaliert horizontal.

## Dimensionierung

Dimensionen Zielsystem Prototyp:
- Speicheranforderungen: Speichern Nur Text: 15 GB/Tag -> = 5 TB/Jahr
Redundanz -> RF = 3 -> = 15 TB /Jahr -> 8 Festplatten a 2 TB -> 1 Unit
- CPU Anforderungen: 4 Kerne mit 8 Threats (1.8 Ghz Base Clock) = 2000 Tweets/ Sekunde
Höchstleistung = 10 000 Tweets / Sekunde
Abdeckung von Systemausfällen und Anstieg der Nutzer = 20 000 Tweets/ Sekunden -> 40 Kerne mit 8 Threats (1.8 Ghz Base Clock)

Dimensionen Zielsystem kommerzielles System:
- CPU Anforderungen
Prototyp: 4 Kerne mit 8 Threats (1.8 Ghz Base Clock) = 2000 Tweets/ Sekunde
Anforderung: Durchschnitt 6000 Tweets/ Sekunde
Höchster Messtand: 10 000 Tweets/ Sekunde
Abdeckung von Systemausällen = 20 000 Tweets/ Sekunden
Weiterentwicklung des Systems (Analyse von Videos, Bilder, andere Sprachen, etc.)
->x25 Prototypleistung Leistung
1000 Kerne mit 8 Threats (1.8 Ghz Base Clock)
- Speicher Anforderung
Twitter Daten Upload 12 TB/Tag
->4.5 PB/Jahr
45Unit x 8 Festplatten (2TB) = 720 TB
 -> 3 Racks a 45 Units für 1 Jahr


## Fehlertoleranz
Umgang mit Fehlern in der Analyse: 
-  Implementierung von Mechanismen zur Behandlung von Ausnahmen im Code, z. B. Try-Except-Blöcke
-  Implementierung eines Meldesystems und die manuelle Überprüfung der Fehler können die Falsch-Positiv sowie Falsch-Negativen Ereignisse behoben werden

Wie kann Fehlertoleranz in unserem Prototyp sichergestellt werden?
- Die Ausführung des Trainings und der Live-Inferenz sind fehlertolerant, da hier auf die Spark Infrastruktur aufgesetzt wird. Sollten Knoten während der Berechnung ausfallen, wird die Aufgaben auf einem anderen Knoten erneut zur Ausführung gebracht.
- Daten werden während der Spark zur Fehlertoleranz im Cluster repliziert. Dies trägt zur Fehlertoleranz im Falle von Knotenausfällen bei Die verlorenen Daten können mithilfe von nativen Funktionen von Spark wie z.B. dem Lineage Graph, wiederhergestellt werden.


## Anmerkungen zu unserem Prototyp

Abkürzungen des Prototyps:
- Limitierte Integration in Systemlandschaft
- Keine Analye von Videos, Bildern und Tonspuren
- Analyse nur auf Tweets in englischer Sprache
- Tainingsdatensatz ist in Größe limitiert

Echtzeitfähigkeit des Prototyps:
Durch 4 Kerne mit 8 Threats (1.8 Ghz Base Clock) führt zu einem Durchsatz von 2000 Tweets/Sekunde. Dies stellt bereits einen erheblichen Anteil dar, wenn man dies mit den durchschnittlichen Menge von 6000 Tweets/ Sekunde vergleicht. Jedoch handelt es sich hierbei um Tweets in reiner Textform. Für eine Live-Analyse, mit Metadaten sowie Tweets welche Bilder und Ton beinhalten, müsste das System ausgebaut werden. Dies könnte durch die Eigenschaften von Spark Streaming mit einer horizontalen Skalierung durchgeführt werden.

Durch das Entfernen von I/O-Aufgaben (z.B. persistieren von Hate-Speech in parquet-Dateien) konnte der Durchsatz auf über 6000 Tweets/Sekunde gesteigert werden. Somit wäre für dieses triviale Modell schon ein Server Echtzeitfähig.

In [2]:
import findspark

findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

import time
from pyspark.ml import PipelineModel

import os


MAX_MEMORY = "6g"


spark = SparkSession \
    .builder \
    .appName("live_hate_prediction") \
    .master("local[8]") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.3.1") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

print("Apache Spark version: ", spark.version)
spark

Apache Spark version:  3.3.1


## Local Streaming mit Parquet
Als Streaming-Quelle für den Prototyp nutzen wir das lokale Filesystem. Hier haben wir ca. 4 Mio. ungesehene Tweets, welche auf Hate-Nachrichten überprüft werden sollen. 
Dafür sind die Tweets mit Metadaten im Parquet-Format gespeichert (ca 5 GB Rohdaten).
Folgende Schritte sind hierfür notwendig:
- Einlesen des Daten-Schemas
- Erstellen eines ReadStreams auf den Daten (die maximale Anzahl der Files pro Trigger wird auf zwei begrenzt.)

Local Spark Environment (3.3.1)
- 4 Cores, 8 Threats (1.8 Ghz Base Clock)
- 8 GB RAM (shared with OS)
- SSD 

In [3]:
spark_df = spark.read.parquet('C://data/twitter-stream-parquet-complete/')
dataSchema = spark_df.schema

print(f" Anzahl der Partitions: {spark_df.rdd.getNumPartitions()}")

 Anzahl der Partitions: 50


In [4]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 2).parquet('C://data/twitter-stream-parquet-complete/')

Inferenz Pipeline laden
- Die zuvor erstelle Inferenzpipeline wird in den Hauptspeichergeladen.

In [5]:
trained_pipeline = PipelineModel.load("../models/mllib_model_nb")

### Local Batch Processing

Die Daten werden in Batches unterteilt und jeder Batch wird einzeln und parallel mit den Sparkoperationen verarbeitet. Die Ergebnisse der einzelnen Batches werden zum Endergebnis kombiniert. 

Folgende Methoden werden auf den Daten ausgeführt:
- Submethode clean_text wird auf jedem Micro-Batch ausgeführt und bereinigt die Texte (z.B. Groß- und Kleinschreibung, Sondernzeichen, ...)
- Methode batch_hate_interference wird auf jedem Micro-Batch ausgeführt und ruft Submethode clean_text auf, führt die Inferenz-Pipeline aus und gibt die Anzahl der identifizierten Hate-Tweets aus Alternativ könnten die Tweets für eine weitere Verabeitung gespeichert werden. 

In [6]:
def clean_text(c):
  c = lower(c)
  c = regexp_replace(c, "(https?\://)\S+", "") # Remove links
  c = regexp_replace(c, "(\\n)|\n|\r|\t", "") # Remove CR, tab, and LR
  c = regexp_replace(c, "(?:(?:[0-9]{2}[:\/,]){2}[0-9]{2,4})", "") # Remove dates
  c = regexp_replace(c, "@([A-Za-z0-9_]+)", "") # Remove usernames
  c = regexp_replace(c, "[0-9]", "") # Remove numbers
  c = regexp_replace(c, "\:|\/|\#|\.|\?|\!|\&|\"|\,", "") # Remove symbols
  return c

In [7]:
def batch_hate_inference(df, test):

    print(f'Iter: {test}')

    # Clean Text Columns
    df = df.withColumn("text", clean_text(col("text")))

    # Predict on Batch
    final_df = trained_pipeline.transform(df).select('text','prediction')

    # Handling for data display detected times:
    hate = final_df.filter(final_df['prediction'] == 1.0)

    # Durch das Entfernen von I/O-Zugriffen konnte der Durchsatz von 2000 Tweets/Sekunde auf über 6000 Tweets/Sekunde gesteiger werden.
    # hate.write.mode('append').parquet('../data/parquet_output')
    times_hate = hate.count()
    print(f'Times hate detected: {times_hate}')

    not_hate = final_df.filter(final_df['prediction'] == 0.0)
    times_not_hate = not_hate.count()
    print(f'Times not hate detected: {times_not_hate}')


- Streaming-Methoden Foreach und ForeachBatch ermöglichen, dass eigene Logik auf den Stream ausgeführt werden kann. 
    - Trigger: availableNow ist nur für den Demo-Modus, beendet den Stream nachdem alle Verfügbaren Daten abgearbeitet wurden.
    - Start: Beginnt Ausführung 

In [8]:
batchHateInference = streaming.writeStream.foreachBatch(batch_hate_inference).trigger(availableNow=True).start() #

Iter: 0
Times hate detected: 1499
Times not hate detected: 32784
Iter: 1
Times hate detected: 1544
Times not hate detected: 34163
Iter: 2
Times hate detected: 1634
Times not hate detected: 35004
Iter: 3
Times hate detected: 1732
Times not hate detected: 36901
Iter: 4
Times hate detected: 1576
Times not hate detected: 35071
Iter: 5
Times hate detected: 1676
Times not hate detected: 35051
Iter: 6
Times hate detected: 1560
Times not hate detected: 35368
Iter: 7
Times hate detected: 1653
Times not hate detected: 34952
Iter: 8
Times hate detected: 1631
Times not hate detected: 35569
Iter: 9
Times hate detected: 1632
Times not hate detected: 34999
Iter: 10
Times hate detected: 1594
Times not hate detected: 34570
Iter: 11
Times hate detected: 1666
Times not hate detected: 34812
Iter: 12
Times hate detected: 1553
Times not hate detected: 34291
Iter: 13
Times hate detected: 1613
Times not hate detected: 33975
Iter: 14
Times hate detected: 1600
Times not hate detected: 33626
Iter: 15
Times hate 

Daten zu Diagnose und Protokoll der Ausführung:
- Mit dem aktuellen Setup (lokales Spark) können im Durschnitt ca. 7000 Tweets / Sekunde untersucht werden.
- Durch Spark Streaming kann der Workload horizontal skaliert werden.

In [13]:
batchHateInference.recentProgress

[{'id': '8ee96ceb-ee4e-4af7-84d3-f50444779b59',
  'runId': 'af6bf608-c563-46bd-bd0e-6b84828edee6',
  'name': None,
  'timestamp': '2023-02-03T08:05:23.108Z',
  'batchId': 20,
  'numInputRows': 70028,
  'inputRowsPerSecond': 6288.998652896273,
  'processedRowsPerSecond': 6624.538832655378,
  'durationMs': {'addBatch': 9662,
   'getBatch': 42,
   'latestOffset': 315,
   'queryPlanning': 3,
   'walCommit': 254},
  'stateOperators': [],
  'sources': [{'description': 'FileStreamSource[file:/C:/data/twitter-stream-parquet-complete]',
    'startOffset': {'logOffset': 19},
    'endOffset': {'logOffset': 20},
    'latestOffset': None,
    'numInputRows': 70028,
    'inputRowsPerSecond': 6288.998652896273,
    'processedRowsPerSecond': 6624.538832655378}],
  'sink': {'description': 'ForeachBatchSink', 'numOutputRows': -1}},
 {'id': '8ee96ceb-ee4e-4af7-84d3-f50444779b59',
  'runId': 'af6bf608-c563-46bd-bd0e-6b84828edee6',
  'name': None,
  'timestamp': '2023-02-03T08:05:33.680Z',
  'batchId': 21,

In [14]:
batchHateInference.status

{'message': 'Stopped', 'isDataAvailable': False, 'isTriggerActive': False}

## Online Straming mit Kafka
Um die ständig neuen Tweets möglichst realitätsgetreu zu simulieren, wurde eine Umgebung mit Kafka als Message-Broker aufgebaut. Ein Python-Skript (kafka_producer.py) erzeugt die Tweets mit zufälligem Text und sendet diese an ein Topic auf dem Kafka-Server. Der Kafka-Server wurde in einer Docker-Umgebung (Docker-Compose) aufgebaut (Ausführung mit Docker Compase mit dem hinterlegten docker-compose.yml). 

Die Prediction Pipelines in Spark Streaming hören auf das Kafak-Topic und führen die MLLib Inferenz Pipeline auf die neu ankommenden Daten aus. Es wurden zwei Varianten der MLLib Pipeline implementiert:
Batch-Prediction: live_hate_prediction_batch.py 
Live-Prediction (Continuous-Mode): live_hate_prediction_live.py

Local Spark Environment (3.3.1)
- 4 Cores, 8 Threats (1.8 Ghz Base Clock)
- 8 GB RAM (shared with OS)
- SSD 

Die Ergebnisse werden im Folgenden Abschnitt beschrieben.

## Batch Live Prediction (live_hate_prediction_batch)

Durch den Kafka Stream werden ca. 6000 Tweets / Sekunde klassifiziert als Hate = 1, Note Hate = 0. Im Vergleich mit dem Durchschnitt von Twitter, welcher 6000 Tweets / Sekunde beträgt, kann die Anwendung mit der aktuellen Komplexität und den oben gegebenen Leistungsspezifikationen den aktuellen Twitter Traffic bewältigen. Als Zeitintervall für die Ausführung wurden 2 Sekunden festgelegt.

![Alt text](../../image2.png)
![Alt text](../../image.png)

Ein genauerer Blick auf die Spark Web UI ziegt, dass die Spark Jobs sequentiell bearbeitet werden, da der Computer über aussreichende Leistung verfügt. Ein erhöhen des Stream-Output konnte nicht umgesetzt werden. Um die Anwendung an ihr Limit zu bringen, müsste entweder der Input an Daten gesteigert werden oder die Komplexität der Analyse steigen.

![Alt text](../../image4.png)

## Continuous Live Prediction (live_hate_prediction_live)

Für die Ausführung der Continous Live Predictions wurde der Continous-Mode ausgewählt. Dieser verarbeitet die Tweets, sobald diese zur Verfügung stehen.

Die Batch Live Prediction des Kafka Streams ist ein ununterbrochener Job. Somit sind keine Statiken über die Durchführung in der Spark UI ersichtlich. Jedoch zeigt die Ausgabe in der Konsole, dass das Programm korrekt ausgeführt wird.



![Alt text](../../image00.png)
![Alt text](../../image5.png)

